In [198]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import os
import networkx as nx

def construire_connexions_geo_MST_par_ligne():
    """
    Construit les connexions entre stations de chaque ligne :
    - Utilise la distance géographique réelle (geodesic)
    - Construit un graphe par ligne
    - Applique un arbre couvrant minimal (MST) pour connecter les stations les plus proches
    Produit un CSV enrichi avec temps estimé, pollution moyenne, coordonnées, et ligne.
    """
    base_dir = os.path.dirname(os.getcwd())
    path_stations = os.path.join(base_dir, "Data", "processed", "stations_qgis.csv")
    df = pd.read_csv(path_stations)

    # Nettoyage
    df.columns = df.columns.str.strip()
    df['Nom de la Station'] = df['Nom de la Station'].str.strip()
    df['Nom de la ligne'] = df['Nom de la ligne'].str.strip()
    df['Niveau de pollution'] = df['Niveau de pollution'].str.lower()
    df = df.drop_duplicates(subset=['Nom de la Station', 'Nom de la ligne'])

    pollution_map = {'faible': 1, 'moyenne': 2, 'élevée': 3, 'eleve': 3}
    connexions = []

    for ligne, group in df.groupby("Nom de la ligne"):
        stations = group.reset_index(drop=True)
        G = nx.Graph()

        # Ajouter tous les sommets
        for _, row in stations.iterrows():
            G.add_node(row['Nom de la Station'], **row.to_dict())

        # Ajouter toutes les arêtes possibles avec la distance
        for i in range(len(stations)):
            for j in range(i + 1, len(stations)):
                s1 = stations.iloc[i]
                s2 = stations.iloc[j]
                coord1 = (s1['stop_lat'], s1['stop_lon'])
                coord2 = (s2['stop_lat'], s2['stop_lon'])
                distance_km = geodesic(coord1, coord2).km
                G.add_edge(s1['Nom de la Station'], s2['Nom de la Station'], weight=distance_km)

        # Calcul du MST (arbre couvrant minimal)
        mst = nx.minimum_spanning_tree(G, weight='weight')

        for u, v, d in mst.edges(data=True):
            s1, s2 = u, v
            n1, n2 = G.nodes[s1], G.nodes[s2]

            distance_km = d['weight']
            temps = round((distance_km / 65) * 60, 2)

            p1 = pollution_map.get(n1['Niveau de pollution'], np.nan)
            p2 = pollution_map.get(n2['Niveau de pollution'], np.nan)
            pollution_moyenne = round((p1 + p2) / 2, 2) if not np.isnan(p1) and not np.isnan(p2) else np.nan

            connexions.append({
                'station1': s1,
                'station2': s2,
                'connected': True,
                'temps_trajet_min': temps,
                'pollution_moyenne': pollution_moyenne,
                'lat_station1': n1['stop_lat'],
                'lon_station1': n1['stop_lon'],
                'lat_station2': n2['stop_lat'],
                'lon_station2': n2['stop_lon'],
                'ligne_metro': ligne
            })

    # Export CSV
    df_connexions = pd.DataFrame(connexions)
    output_path = os.path.join(base_dir, "Data", "processed", "trajets_stations_pollution.csv")
    df_connexions.to_csv(output_path, index=False)
    print(f"Fichier exporté avec MST par ligne vers : {output_path}")
    return df_connexions

# Exécution
if __name__ == "__main__":
    construire_connexions_geo_MST_par_ligne()


Fichier exporté avec MST par ligne vers : c:\projets\Pollution_Data_Analysis_Transport_Stations\Data\processed\trajets_stations_pollution.csv


En suivant les lignes de métros et en calculant la distance avec les stations, nous avons pu établir les connections des stations. Cepdendant il reste quelques erreurs que nous devont régler manuellement

In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import os

# Charger le CSV existant
base_dir = os.path.dirname(os.getcwd())
csv_path = os.path.join(base_dir, "Data", "processed", "trajets_stations_pollution.csv")
df = pd.read_csv(csv_path)

# Nettoyer les colonnes
df['station1'] = df['station1'].str.strip()
df['station2'] = df['station2'].str.strip()

# Supprimer le lien Guy Môquet ↔ Brochant (dans les deux sens)
df = df[~(
    ((df['station1'] == 'Guy Môquet') & (df['station2'] == 'Brochant')) |
    ((df['station1'] == 'Brochant') & (df['station2'] == 'Guy Môquet'))
)]

# Créer une nouvelle ligne pour La Fourche ↔ Guy Môquet
# → Tu peux récupérer les coordonnées depuis df directement :
stations = pd.concat([
    df[df['station1'] == 'La Fourche'],
    df[df['station2'] == 'La Fourche'],
    df[df['station1'] == 'Guy Môquet'],
    df[df['station2'] == 'Guy Môquet']
])

def get_coords(station_name):
    row = stations[(stations['station1'] == station_name) | (stations['station2'] == station_name)].iloc[0]
    lat = row['lat_station1'] if row['station1'] == station_name else row['lat_station2']
    lon = row['lon_station1'] if row['station1'] == station_name else row['lon_station2']
    return lat, lon

lat1, lon1 = get_coords('La Fourche')
lat2, lon2 = get_coords('Guy Môquet')

# Calculer distance et pollution
distance_km = geodesic((lat1, lon1), (lat2, lon2)).km
temps = round((distance_km / 65) * 60, 2)
pollution_moyenne = 2.0  # valeur par défaut ou à affiner si besoin

# Ajouter la nouvelle ligne
nouvelle_connexion = pd.DataFrame([{
    'station1': 'La Fourche',
    'station2': 'Guy Môquet',
    'connected': True,
    'temps_trajet_min': temps,
    'pollution_moyenne': pollution_moyenne,
    'lat_station1': lat1,
    'lon_station1': lon1,
    'lat_station2': lat2,
    'lon_station2': lon2,
    'ligne_metro': 'Métro 13'
}])

df = pd.concat([df, nouvelle_connexion], ignore_index=True)

# Nettoyage noms
df['station1'] = df['station1'].str.strip()
df['station2'] = df['station2'].str.strip()

# Supprimer le lien Kremlin-Bicêtre ↔ Porte d'Italie
df = df[~(
    ((df['station1'] == 'Le Kremlin-Bicêtre') & (df['station2'] == "Porte d'Italie")) |
    ((df['station2'] == 'Le Kremlin-Bicêtre') & (df['station1'] == "Porte d'Italie"))
)]

# Chercher les coordonnées existantes
stations_kb_mb = pd.concat([
    df[df['station1'].isin(['Le Kremlin-Bicêtre', 'Maison Blanche'])],
    df[df['station2'].isin(['Le Kremlin-Bicêtre', 'Maison Blanche'])]
])

def get_coords(station_name):
    row = stations_kb_mb[(stations_kb_mb['station1'] == station_name) | (stations_kb_mb['station2'] == station_name)].iloc[0]
    lat = row['lat_station1'] if row['station1'] == station_name else row['lat_station2']
    lon = row['lon_station1'] if row['station1'] == station_name else row['lon_station2']
    return lat, lon

lat1, lon1 = get_coords('Le Kremlin-Bicêtre')
lat2, lon2 = get_coords('Maison Blanche')

# Calcul distance et temps
distance_km = geodesic((lat1, lon1), (lat2, lon2)).km
temps = round((distance_km / 65) * 60, 2)
pollution_moyenne = 2.0  # valeur par défaut si pas de donnée fine

# Ajouter la nouvelle ligne
nouvelle_connexion_kb_mb = pd.DataFrame([{
    'station1': 'Le Kremlin-Bicêtre',
    'station2': 'Maison Blanche',
    'connected': True,
    'temps_trajet_min': temps,
    'pollution_moyenne': pollution_moyenne,
    'lat_station1': lat1,
    'lon_station1': lon1,
    'lat_station2': lat2,
    'lon_station2': lon2,
    'ligne_metro': 'Métro 7'
}])

df = pd.concat([df, nouvelle_connexion_kb_mb], ignore_index=True)

# Suppression d’un lien existant
df = df[~(
    ((df['station1'] == 'Porte d’Auteuil') & (df['station2'] == 'Michel-Ange - Molitor')) |
    ((df['station2'] == 'Porte d’Auteuil') & (df['station1'] == 'Michel-Ange - Molitor'))
)]

# Recherche des coordonnées pour 2 stations à connecter
stations_m10 = pd.concat([
    df[df['station1'].isin([
        'Boulogne Jean Jaurès', 'Michel-Ange - Molitor',
        'Porte d’Auteuil', 'Michel-Ange - Auteuil'
    ])],
    df[df['station2'].isin([
        'Boulogne Jean Jaurès', 'Michel-Ange - Molitor',
        'Porte d’Auteuil', 'Michel-Ange - Auteuil'
    ])]
])

def get_coords_m10(station_name):
    row = stations_m10[(stations_m10['station1'] == station_name) | (stations_m10['station2'] == station_name)].iloc[0]
    lat = row['lat_station1'] if row['station1'] == station_name else row['lat_station2']
    lon = row['lon_station1'] if row['station1'] == station_name else row['lon_station2']
    return lat, lon

# Ajout : Boulogne Jean Jaurès ↔ Michel-Ange - Molitor
lat1, lon1 = get_coords_m10('Boulogne Jean Jaurès')
lat2, lon2 = get_coords_m10('Michel-Ange - Molitor')
distance_km = geodesic((lat1, lon1), (lat2, lon2)).km
temps = round((distance_km / 65) * 60, 2)

ligne = 'Métro 10'
pollution_moyenne = 2.0

nouvelle_connexion_m10a = pd.DataFrame([{
    'station1': 'Boulogne Jean Jaurès',
    'station2': 'Michel-Ange - Molitor',
    'connected': True,
    'temps_trajet_min': temps,
    'pollution_moyenne': pollution_moyenne,
    'lat_station1': lat1,
    'lon_station1': lon1,
    'lat_station2': lat2,
    'lon_station2': lon2,
    'ligne_metro': ligne
}])

# Ajout : Porte d’Auteuil ↔ Michel-Ange - Auteuil
lat1, lon1 = get_coords_m10('Porte d’Auteuil')
lat2, lon2 = get_coords_m10('Michel-Ange - Auteuil')
distance_km = geodesic((lat1, lon1), (lat2, lon2)).km
temps = round((distance_km / 65) * 60, 2)

nouvelle_connexion_m10b = pd.DataFrame([{
    'station1': 'Porte d’Auteuil',
    'station2': 'Michel-Ange - Auteuil',
    'connected': True,
    'temps_trajet_min': temps,
    'pollution_moyenne': pollution_moyenne,
    'lat_station1': lat1,
    'lon_station1': lon1,
    'lat_station2': lat2,
    'lon_station2': lon2,
    'ligne_metro': ligne
}])

# Ajout dans df
df = pd.concat([df, nouvelle_connexion_m10a, nouvelle_connexion_m10b], ignore_index=True)


IndexError: single positional indexer is out-of-bounds

In [181]:
import pandas as pd
import numpy as np
import networkx as nx

def construire_graphe_transport_pollution(path_csv="../Data/processed/trajets_stations_pollution.csv"):
    """
    Construit un graphe non orienté des stations de métro à partir du fichier enrichi,
    en utilisant les colonnes :
    - station1, station2
    - temps_trajet_min
    - pollution_moyenne

    Le graphe résultant a les arêtes pondérées par temps et pollution.
    """
    # Chargement des données
    df = pd.read_csv(path_csv)

    # Création du graphe
    G = nx.Graph()
    aretes_vues = set()

    for _, row in df.iterrows():
        if row.get('connected', True) and pd.notnull(row['temps_trajet_min']):
            station1 = row['station1']
            station2 = row['station2']
            edge_key = tuple(sorted([station1, station2]))

            if edge_key not in aretes_vues:
                pollution_moyenne = row.get('pollution_moyenne', np.nan)

                G.add_edge(
                    station1,
                    station2,
                    temps=row['temps_trajet_min'],
                    pollution=pollution_moyenne
                )
                aretes_vues.add(edge_key)

    print(f"Graphe construit avec {G.number_of_nodes()} sommets et {G.number_of_edges()} arêtes.")
    return G

# Exemple d'utilisation
if __name__ == "__main__":
    G = construire_graphe_transport_pollution()


Graphe construit avec 320 sommets et 376 arêtes.


In [168]:
print(f"Nombre de sommets : {G.number_of_nodes()}")
print(f"Nombre d’arêtes : {G.number_of_edges()}")
print("Exemple d'arête :")
for u, v, data in list(G.edges(data=True))[:5]:
    print(f"{u} -> {v}, temps: {data['temps']}, pollution: {data['pollution']}")


Nombre de sommets : 320
Nombre d’arêtes : 376
Exemple d'arête :
Château de Vincennes -> Bérault, temps: 0.78, pollution: 1.0
Bérault -> Saint-Mandé, temps: 0.7, pollution: 1.5
Châtelet -> Louvre - Rivoli, temps: 0.37, pollution: 1.5
Châtelet -> Hôtel de Ville, temps: 0.47, pollution: 1.5
Châtelet -> Pyramides, temps: 1.02, pollution: 1.5


In [170]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

# Liste des stations
stations = sorted(G.nodes())

# Widget de sélection
station_selector = widgets.Dropdown(
    options=stations,
    description="Station:",
    layout=widgets.Layout(width='50%')
)

# Fonction d'affichage des voisins
def afficher_voisins(station):
    voisins = list(G.neighbors(station))
    if not voisins:
        display(Markdown(f"**Aucune connexion depuis {station}.**"))
        return

    lignes = []
    for voisin in voisins:
        data = G[station][voisin]
        lignes.append(f"- Vers **{voisin}** : {data['temps']} min, pollution moyenne = {data['pollution']}")
    
    display(Markdown(f"### Connexions depuis **{station}** :\n" + "\n".join(lignes)))

# Lier le widget à la fonction
widgets.interact(afficher_voisins, station=station_selector)


interactive(children=(Dropdown(description='Station:', layout=Layout(width='50%'), options=('Abbesses', 'Aimé …

<function __main__.afficher_voisins(station)>

In [171]:
import pandas as pd
import plotly.graph_objects as go
import os

# Chemins des fichiers
base_dir = os.path.dirname(os.getcwd())
trajets_path = os.path.join(base_dir, "Data", "processed", "trajets_stations_pollution.csv")
stations_path = os.path.join(base_dir, "Data", "processed", "stations_qgis.csv")

# Chargement des données
df_connexions = pd.read_csv(trajets_path)
df_stations = pd.read_csv(stations_path)

# On garde les connexions valides
df_connexions = df_connexions[df_connexions['connected'] == True]

# Dictionnaire couleur par ligne de métro
lignes_uniques = df_connexions['ligne_metro'].dropna().unique()
couleurs = px.colors.qualitative.Plotly
couleur_par_ligne = {ligne: couleurs[i % len(couleurs)] for i, ligne in enumerate(sorted(lignes_uniques))}

# Traces des segments de métro (lignes)
segments = []
for _, row in df_connexions.iterrows():
    lat1, lon1 = row['lat_station1'], row['lon_station1']
    lat2, lon2 = row['lat_station2'], row['lon_station2']
    ligne = row['ligne_metro']
    couleur = couleur_par_ligne.get(ligne, "gray")

    segments.append(go.Scattermapbox(
        lat=[lat1, lat2],
        lon=[lon1, lon2],
        mode='lines',
        line=dict(width=2, color=couleur),
        hoverinfo='text',
        text=f"{row['station1']} ↔ {row['station2']} ({ligne})",
        showlegend=False
    ))

# Traces des stations
stations_unique = df_stations.drop_duplicates(subset="Nom de la Station")
station_trace = go.Scattermapbox(
    lat=stations_unique["stop_lat"],
    lon=stations_unique["stop_lon"],
    mode='markers',
    marker=dict(size=7, color="black"),
    text=stations_unique["Nom de la Station"],
    hoverinfo="text",
    name="Stations"
)

# Création de la figure
fig = go.Figure(data=[station_trace] + segments)

# Mise en page Mapbox
fig.update_layout(
    title="Carte interactive des stations de métro et connexions",
    mapbox=dict(
        style="carto-positron",
        zoom=11,
        center=dict(lat=48.8566, lon=2.3522)  # Paris
    ),
    height=700,
    margin=dict(l=10, r=10, t=40, b=10)
)

fig.show()


C:\Users\Administrateur\AppData\Local\Temp\ipykernel_23764\352706971.py:30: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

C:\Users\Administrateur\AppData\Local\Temp\ipykernel_23764\352706971.py:42: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [111]:
def chemin_min_pollution(G, depart, arrivee, temps_max):
    from heapq import heappush, heappop

    # Vérifications de base
    if depart not in G.nodes:
        return f"Station de départ '{depart}' introuvable dans le graphe."
    if arrivee not in G.nodes:
        return f"Station d'arrivée '{arrivee}' introuvable dans le graphe."

    try:
        _ = nx.shortest_path(G, source=depart, target=arrivee)
    except nx.NetworkXNoPath:
        return f"Aucun chemin possible entre '{depart}' et '{arrivee}'."

    # File de priorité : (pollution cumulée, temps cumulé, station actuelle, chemin parcouru)
    file = [(0, 0, depart, [])]
    visited = {}

    while file:
        pollution_cumulee, temps_cumule, current, chemin = heappop(file)
        chemin = chemin + [current]

        # Éviter les redondances
        if current in visited and visited[current] <= temps_cumule:
            continue
        visited[current] = temps_cumule

        # Si destination atteinte dans les temps
        if current == arrivee and temps_cumule <= temps_max:
            pollution_moyenne = pollution_cumulee / len(chemin)
            return {
                'chemin': chemin,
                'temps_total': round(temps_cumule, 2),
                'pollution_moyenne': round(pollution_moyenne, 2)
            }

        for voisin in G.neighbors(current):
            data = G[current][voisin]

            # Pollution avec valeur par défaut
            pollution = data.get('pollution', 999)
            if pd.isna(pollution):
                pollution = 999

            # Temps avec vérification
            temps = data.get('temps')
            if temps is None:
                continue

            nouveau_temps = temps_cumule + temps
            if nouveau_temps > temps_max:
                continue

            heappush(file, (
                pollution_cumulee + pollution,
                nouveau_temps,
                voisin,
                chemin
            ))

    return f"Aucun chemin ne respecte la contrainte de temps maximale de {temps_max} minutes entre '{depart}' et '{arrivee}'."


In [112]:
# Exemple avec affichage du résultat
resultat = chemin_min_pollution(G, "République", "Gare de Lyon", 20)
if isinstance(resultat, dict):
    display(pd.DataFrame([resultat]))
else:
    print(resultat)

,chemin,temps_total,pollution_moyenne
0,"[République, Commerce, Boucicaut, Strasbourg - Saint-Denis, Concorde, Palais Royal - Musée du Louvre, Gare de Lyon]",17.38,856.29


In [92]:
def tester_chemins_pollution(station_depart, station_arrivee):
    """
    Teste les chemins entre deux stations selon différents seuils de pollution.
    Affiche un tableau avec :
    - Seuil utilisé
    - Chemin trouvé ou non
    - Trajet
    - Pollution sur le trajet
    """
    resultats_txt = []
   
    for seuil in [1, 2, 3]:
        existe, chemin, pollution_chemin = chemin_pollution_txt(G_txt, station_depart, station_arrivee, seuil)
       
        pollution_textuelle = ["faible" if p == 1 else "moyenne" if p == 2 else "élevée" for p in pollution_chemin]
       
        if   seuil == 1: description_seuil = "faible"
        elif seuil == 2: description_seuil = "faible ou moyenne"
        else:            description_seuil = "faible, moyenne ou élevée"
       
        resultats_txt.append({
            "Seuil de pollution autorisé": description_seuil,
            "Chemin trouvé": "Oui" if existe else "Non",
            "Trajet": " -> ".join(chemin) if existe else "Aucun",
            "Pollution sur le trajet": pollution_textuelle if existe else "N/A"
        })
    
    return resultats_txt

In [94]:
# Exemple d'utilisation
resultats = tester_chemins_pollution("Gare de Lyon", "République")
if isinstance(resultats, list) and all(isinstance(r, dict) for r in resultats):
    display(pd.DataFrame(resultats))
else:
    print(resultats)

,Seuil de pollution autorisé,Chemin trouvé,Trajet,Pollution sur le trajet
0,faible,Non,Aucun,N/A
1,faible ou moyenne,Oui,Gare de Lyon -> Palais Royal - Musée du Louvre -> Concorde -> Strasbourg - Saint-Denis -> Boucicaut -> Commerce -> République,"[moyenne, moyenne, moyenne, moyenne, faible, faible, moyenne]"
2,"faible, moyenne ou élevée",Oui,Gare de Lyon -> Palais Royal - Musée du Louvre -> Concorde -> Strasbourg - Saint-Denis -> Boucicaut -> Commerce -> République,"[moyenne, moyenne, moyenne, moyenne, faible, faible, moyenne]"


In [98]:
# Charger les données avec les coordonnées GPS
df_connexions = pd.read_csv("../Data/processed/trajets_stations_pollution.csv")

# Ajouter lat/lon aux nœuds de G_txt à partir de station1
for _, row in df_connexions.iterrows():
    s1, s2 = row['station1'], row['station2']

    if s1 in G_txt.nodes:
        G_txt.nodes[s1]['lat'] = row['lat_station1']
        G_txt.nodes[s1]['lon'] = row['lon_station1']
    if s2 in G_txt.nodes:
        G_txt.nodes[s2]['lat'] = row['lat_station2']
        G_txt.nodes[s2]['lon'] = row['lon_station2']

# Vérification rapide (facultative)
print(f"✅ Coordonnées ajoutées à {len(G_txt.nodes)} stations.")


✅ Coordonnées ajoutées à 216 stations.


In [113]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Préparation des stations disponibles
stations_list = sorted(set(G_txt.nodes))

# Widgets
station_depart_widget = widgets.Dropdown(options=stations_list, description="Départ")
station_arrivee_widget = widgets.Dropdown(options=stations_list, description="Arrivée")
fonction_widget = widgets.RadioButtons(
    options=["tester_chemins_pollution", "chemin_min_pollution"],
    description="Fonction:"
)
temps_max_widget = widgets.IntSlider(value=15, min=1, max=60, step=1, description="Temps max")

def afficher_resultats(depart, arrivee, fonction, temps_max):
    import itertools
    from matplotlib import cm

    print(f"🟢 Départ : {depart}   🔵 Arrivée : {arrivee}")
    print(f"📌 Fonction choisie : {fonction}")

    fig = go.Figure()
    chemins_affiches = False
    all_lats, all_lons = [], []

    if fonction == "tester_chemins_pollution":
        resultats = tester_chemins_pollution(depart, arrivee)
        display(pd.DataFrame(resultats))

        cmap = cm.get_cmap('tab10')
        couleurs = itertools.cycle([
            f'rgb({int(r*255)},{int(g*255)},{int(b*255)})'
            for r, g, b, _ in cmap(np.linspace(0, 1, 10))
        ])

        for resultat in resultats:
            if resultat["Chemin trouvé"] == "Oui":
                chemin = resultat["Trajet"].split(" -> ")
                couleur = next(couleurs)

                for i in range(len(chemin) - 1):
                    n1, n2 = chemin[i], chemin[i+1]
                    lat1, lon1 = G_txt.nodes[n1].get('lat'), G_txt.nodes[n1].get('lon')
                    lat2, lon2 = G_txt.nodes[n2].get('lat'), G_txt.nodes[n2].get('lon')
                    if lat1 and lon1 and lat2 and lon2:
                        fig.add_trace(go.Scattermapbox(
                            lat=[lat1, lat2],
                            lon=[lon1, lon2],
                            mode='lines',
                            line=dict(width=2, color=couleur),
                            showlegend=False
                        ))
                        all_lats += [lat1, lat2]
                        all_lons += [lon1, lon2]

                coords = []
                for n in chemin:
                    if n in G_txt.nodes:
                        lat = G_txt.nodes[n].get('lat')
                        lon = G_txt.nodes[n].get('lon')
                        if lat is not None and lon is not None:
                            coords.append((lat, lon))

                fig.add_trace(go.Scattermapbox(
                    lat=[lat for lat, lon in coords],
                    lon=[lon for lat, lon in coords],
                    mode='markers+text',
                    text=chemin,
                    marker=dict(size=9, color=couleur),
                    name='Chemin'
                ))

                chemins_affiches = True

    else:
        res = chemin_min_pollution(G_txt, depart, arrivee, temps_max)
        if isinstance(res, str):
            print(res)
            return
        display(pd.DataFrame([res]))
        chemin = res['chemin']

        for i in range(len(chemin) - 1):
            n1, n2 = chemin[i], chemin[i+1]
            lat1, lon1 = G_txt.nodes[n1].get('lat'), G_txt.nodes[n1].get('lon')
            lat2, lon2 = G_txt.nodes[n2].get('lat'), G_txt.nodes[n2].get('lon')
            if lat1 and lon1 and lat2 and lon2:
                fig.add_trace(go.Scattermapbox(
                    lat=[lat1, lat2],
                    lon=[lon1, lon2],
                    mode='lines',
                    line=dict(width=3, color='blue'),
                    showlegend=False
                ))
                all_lats += [lat1, lat2]
                all_lons += [lon1, lon2]

        coords = []
        for n in chemin:
            if n in G_txt.nodes:
                lat = G_txt.nodes[n].get('lat')
                lon = G_txt.nodes[n].get('lon')
                if lat is not None and lon is not None:
                    coords.append((lat, lon))

        fig.add_trace(go.Scattermapbox(
            lat=[lat for lat, lon in coords],
            lon=[lon for lat, lon in coords],
            mode='markers+text',
            text=chemin,
            marker=dict(size=10, color='red'),
            name='Chemin optimal'
        ))

        chemins_affiches = True

    if chemins_affiches:
        fig.update_layout(
            mapbox_style="carto-positron",
            mapbox_zoom=11,
            mapbox_center={
                "lat": all_lats[0] if all_lats else 48.85,
                "lon": all_lons[0] if all_lons else 2.35
            },
            height=600,
            margin={"r": 0, "t": 30, "l": 0, "b": 0},
            title="Trajets sur le graphe"
        )
        fig.show()
    else:
        print("❗ Aucun chemin visualisable trouvé pour affichage cartographique.")



# Interface interactive
ui = widgets.VBox([
    station_depart_widget,
    station_arrivee_widget,
    fonction_widget,
    temps_max_widget
])

out = widgets.interactive_output(
    afficher_resultats,
    {
        'depart': station_depart_widget,
        'arrivee': station_arrivee_widget,
        'fonction': fonction_widget,
        'temps_max': temps_max_widget
    }
)

display(ui, out)


Output()